<a href="https://colab.research.google.com/github/manuellasarte/pytorch-deep-learning/blob/main/quickstarts/Search_Grounding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Search Grounding

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Search_Grounding.ipynb"><img src="https://github.com/google-gemini/cookbook/blob/main/images/colab_logo_32px.png?raw=1" />Run in Google Colab</a>
  </td>
</table>

This notebook contains a walkthrough of how to use the search grounding feature in the Gemini API.

In [1]:
!pip install -U google-generativeai>=0.8.3

In [2]:
import google.generativeai as genai
from IPython.display import Markdown, HTML

## Set up your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [3]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Search Grounding

Take a look at the available models from the Gemini API. To perform search grounding, use `'models/gemini-1.5-pro-002'` when calling the API.

In [4]:
for model in genai.list_models():
  if '002' in model.name:
    print(model.name)

models/gemini-1.5-pro-002
models/gemini-1.5-flash-002


`GoogleSearchRetrieval` is a tool to retrieve public web data for grounding, powered by Google. Part of `GoogleSearchRetrieval` is dynamic search retrieval, which uses real time retrieved data to build answers to queries.

Two possible ways to use include `GoogleSearchRetrieval` are to pass in a string or dictionary to the field. First, you will pass in `google_search_retrieval` to the `tools` parameter in `generate_content`.

In [5]:
model = genai.GenerativeModel('models/gemini-1.5-pro-002')
response = model.generate_content(contents="What is the land area of Spain?",
                                  tools='google_search_retrieval')

TooManyRequests: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-002:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).

Take a look at the first response produced by the model.

In [ ]:
Markdown(response.candidates[0].content.parts[0].text)

Now, take a look at the rendered HTML content gathered by the Gemini API.

> Important: If you use search grounding you **must** follow the [requirements outlined here](https://googledevai.devsite.corp.google.com/gemini-api/docs/grounding/search-suggestions?hl=en#requirements), which includes "Display the Search Suggestion exactly as provided" and "Take users directly to the Google Search results page (SRP) when they interact with the Search Suggestion".

In [ ]:
HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content)

You will also receive links that direct you to sources used for your query.

In [ ]:
response.candidates[0].grounding_metadata.grounding_chunks

And supporting segments, showing which parts of the output are quored from which source.

In [ ]:
response.candidates[0].grounding_metadata.grounding_supports

### Customize search grounding parameters

For a dictionary implementation, you don't actually have to pass in `mode` or `dynamic_threshold` key-value pairs. If you don't pass in a value, a default value is created. See the next code cell for this. For the `mode` value, you can pass in one of two specifications:

1. `unspecified`: Always trigger retrieval.
2. `dynamic`: Run retrieval only when system decides it is necessary.

The `dynamic_threshold` is a float value that acts as a threshold on how relevant answers returned should be. You can tweak this parameter depending on how much variation you want in your answer.

In [ ]:
model = genai.GenerativeModel('models/gemini-1.5-pro-002')
response = model.generate_content(contents="What is the land area of Spain?",
                                  tools={"google_search_retrieval": {"dynamic_retrieval_config": {"mode": "MODE_DYNAMIC",
                                                                                                  "dynamic_threshold": 0.5}}})

In [ ]:
Markdown(response.candidates[0].content.parts[0].text)

You can also pass in an empty dictionary. Default values for `mode` and `dynamic_threshold` will be used.

In [ ]:
model = genai.GenerativeModel('models/gemini-1.5-pro-002')
response = model.generate_content(contents="What is the land area of Spain?",
                                  tools={"google_search_retrieval": {}})

In [ ]:
Markdown(response.candidates[0].content.parts[0].text)

## With chat

If you attach the `google_search_retrieval` tool to the model, it can be used in every turn of the chat:

In [ ]:
model = genai.GenerativeModel('models/gemini-1.5-pro-002', tools="google_search_retrieval")
chat = model.start_chat()
result = chat.send_message('what is the area of spain?')

In [ ]:
display(Markdown(result.text))

display(HTML(result.candidates[0].grounding_metadata.search_entry_point.rendered_content))

In [ ]:
result = chat.send_message('what is the google stock price?')


display(Markdown(result.text))

display(HTML(result.candidates[0].grounding_metadata.search_entry_point.rendered_content))

But you can also pass the tool through the `send_message` call.

In [6]:
model = genai.GenerativeModel('models/gemini-1.5-pro-002')
chat = model.start_chat()
result = chat.send_message('what is the area of spain?', tools="google_search_retrieval")

TooManyRequests: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-002:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).